# Concurrency

## goroutines

In [1]:
package main

import (
    "fmt"
    "time"
)

func say(s string) {
    for i := 0; i < 5; i++ {
        time.Sleep(100 * time.Millisecond)
        fmt.Println(s)
    }
}

go say("world")
say("hello")

hello
world
hello
world
hello
world
world
hello
world
hello


## Channels

Channels are a typed conduit through which you can send and receive values with the channel operator, `<-`.

```go
ch <- v    // Send v to channel ch.
v := <-ch  // Receive from ch, and
           // assign value to v.
```

(The data flows in the direction of the arrow.)

Like maps and slices, channels must be created before use:

`ch := make(chan int)`

By default, sends and receives block until the other side is ready. This allows goroutines to synchronize without explicit locks or condition variables.

The example code sums the numbers in a slice, distributing the work between two goroutines. Once both goroutines have completed their computation, it calculates the final result.


In [2]:
func sum(s []int, c chan int) {
    sum := 0
    for _, v := range s {
        sum += v
    }
    c <- sum // send sum to c
}

func main() {
    s := []int{7, 2, 8, -9, 4, 0}

    c := make(chan int)
    go sum(s[:len(s)/2], c)
    go sum(s[len(s)/2:], c)
    x, y := <-c, <-c // receive from c

    fmt.Println(x, y, x+y)
}

main()

-5 17 12


##  Buffered Channels

Channels can be buffered. Provide the buffer length as the second argument to make to initialize a buffered channel:

`ch := make(chan int, 100)`

Sends to a buffered channel block only when the buffer is full. Receives block when the buffer is empty.

Modify the example to overfill the buffer and see what happens.


In [3]:
func main() {
	ch := make(chan int, 2)
	ch <- 1
	ch <- 2
	fmt.Println(<-ch)
	fmt.Println(<-ch)
	ch <- 3
	fmt.Println(<-ch)
	ch <- 4
	ch <- 5
	fmt.Println(<-ch)
}
main()

1 true
2 true
3 true
4 true


## Range and Close

 A sender can close a channel to indicate that no more values will be sent. Receivers can test whether a channel has been closed by assigning a second parameter to the receive expression: after

`v, ok := <-ch`

ok is false if there are no more values to receive and the channel is closed.

The loop for i := range c receives values from the channel repeatedly until it is closed.

Note: Only the sender should close a channel, never the receiver. Sending on a closed channel will cause a panic.

Another note: Channels aren't like files; you don't usually need to close them. Closing is only necessary when the receiver must be told there are no more values coming, such as to terminate a range loop. 

In [4]:
func fibonacci(n int, c chan int) {
    x, y := 0, 1
    for i := 0; i < n; i++ {
        c <- x
        x, y = y, x+y
    }
    close(c)
}

func main() {
    c := make(chan int, 7)
    go fibonacci(cap(c), c)
    for i := range c {
        fmt.Println(i)
    }
}

main()

0
1
1
2
3
5
8


##  Select

The select statement lets a goroutine wait on multiple communication operations.

A select blocks until one of its cases can run, then it executes that case. It chooses one at random if multiple are ready.


In [5]:
func fibonacci(c, quit chan int) {
    x, y := 0, 1
    for { // infinite loop
        select {
        case c <- x:
            x, y = y, x+y
        case <-quit:
            fmt.Println("quit")
            return // end loop
        }
    }
}

func main() {
    c := make(chan int)
    quit := make(chan int)
    go func() { // print 10 integers FROM channel c, then send a zero TO channel quit
        for i := 0; i < 10; i++ {
            fmt.Println(i, <-c)
        }
        quit <- 0
    }()
    /* 
        Start running a fibbonachi that populates C in an inf loop.
        However, func() from above populates 'quit' after it reads first
         10 fib numbers from C
    */
    fibonacci(c, quit) 
}

main()

0 0
1 1
2 1
3 2
4 3
5 5
6 8
7 13
8 21
9 34
quit


##  Default Selection

The default case in a select is run if no other case is ready.

Use a default case to try a send or receive without blocking:

```go
select {
case i := <-c:
    // use i
default:
    // receiving from c would block
}

```


In [6]:
func main() {
    tick := time.Tick(100 * time.Millisecond)
    boom := time.After(500 * time.Millisecond)
    for {
        select {
        case <-tick:
            fmt.Println("tick.")
        case <-boom:
            fmt.Println("BOOM!")
            return
        default:
            fmt.Println("    .")
            time.Sleep(50 * time.Millisecond)
        }
    }
}
main()

    .
    .
tick.
    .
    .
tick.
    .
    .
tick.
    .
    .
tick.
    .
    .
tick.
BOOM!


## Exercise: Equivalent Binary Trees

There can be many different binary trees with the same sequence of values stored in it. For example, here are two binary trees storing the sequence 1, 1, 2, 3, 5, 8, 13.

![](https://tour.golang.org/content/img/tree.png)

A function to check whether two binary trees store the same sequence is quite complex in most languages. We'll use Go's concurrency and channels to write a simple solution.

This example uses the `tree` package, which defines the type:

```go
type Tree struct {
    Left  *Tree
    Value int
    Right *Tree
}
```

1. Implement the Walk function.

2. Test the Walk function.

The function `tree.New(k)` constructs a randomly-structured (but always sorted) binary tree holding the values k, 2k, 3k, ..., 10k.

Create a new channel ch and kick off the walker:

`go Walk(tree.New(1), ch)`

Then read and print 10 values from the channel. It should be the numbers 1, 2, 3, ..., 10.

3. Implement the Same function using Walk to determine whether t1 and t2 store the same values.

4. Test the Same function.

`Same(tree.New(1), tree.New(1))` should return `true`, and `Same(tree.New(1), tree.New(2))` should return `false`.

The documentation for `Tree` can be found [here](https://pkg.go.dev/golang.org/x/tour/tree?tab=doc) : https://pkg.go.dev/golang.org/x/tour/tree?tab=doc 




In [7]:
import (
    "golang.org/x/tour/tree"
    "fmt"
    "time"
)




// warning: could not find package "github.com/gopherdata/gophernotes/imports" in $GOPATH = "/home/david/go"
//          command 'import _b "path/to/some/package"' may not work correctly.



In [48]:
t := tree.New(1)

fmt.Println(t.Left, "<-|", t.Value, "|->", t.Right)

// Walk walks the tree t sending all values
// from the tree to the channel ch.
func Walk(t *tree.Tree, ch chan int) {
    /* For a sorted binary tree, the following order will always yield sorted
       numbers:
           1) walk left tree if not null
           2) send the value to chan
           3) walk righ tree if not null
    */
    
    if t.Left != nil {
        Walk(t.Left, ch)
    }
    
    ch <- t.Value
    
    if t.Right != nil {
        Walk(t.Right, ch)
    }
    
}

ch := make(chan int)
go Walk(t, ch)


for i:=0; i<10; i++ {
    v, _ := <-ch
    fmt.Println(v)
}

((1) 2 ((3) 4 (5))) <-| 6 |-> (7 (8 (9 (10))))
1
2
3
4
5
6
7
8
9
10


In [62]:
// Same determines whether the trees
// t1 and t2 contain the same values.
func Same(t1, t2 *tree.Tree) bool {
    /*
        So, since Walk(t, ch) always sends values to a channel in sorted order,
        we can start our walks and compare channels on every iteration. 
        Those channels should return the same values when read
    */
    fmt.Println(t1)
    fmt.Println(t2)
    
    ch1 := make(chan int)
    ch2 := make(chan int)
    go Walk(t1, ch1)
    go Walk(t2, ch2)


    for i:=0; i<10; i++ {
        a := <- ch1
        b := <- ch2
//         fmt.Println(a, b, a==b)
        if a!=b { // channels diverged, therefore t1 != t2
            return false
        }
    }
    
    // if you are here, channels did not diverge, therefore t1 == t2
    return true
}

In [63]:
Same(tree.New(1), tree.New(1))

((1 ((2) 3)) 4 (5 ((6) 7 ((8) 9 (10)))))
(((1 (2)) 3 (4 ((5 (6)) 7))) 8 ((9) 10))


true

In [64]:
Same(tree.New(3), tree.New(3))

(((3) 6 ((((9) 12 (15 ((18) 21))) 24) 27)) 30)
((3) 6 (((9 ((12) 15)) 18) 21 ((24) 27 (30))))


true

In [65]:
Same(tree.New(1), tree.New(4))

((1) 2 (3 ((4 (5 ((6) 7))) 8 (9 (10)))))
((4) 8 (((12) 16 (20 (24))) 28 (32 ((36) 40))))


false